In [ ]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="AmeSame420",
        database="mydb"
    )

def insert_faction(cursor, name, description):
    sql = "INSERT INTO Faction (name, description) VALUES (%s, %s)"
    cursor.execute(sql, (name, description))

def insert_unit(cursor, faction_id, name, description):
    sql = "INSERT INTO Unit (faction_id, name, description) VALUES (%s, %s, %s)"
    cursor.execute(sql, (faction_id, name, description))

def get_faction_id(cursor, name):
    sql = "SELECT id FROM Faction WHERE name = %s"
    cursor.execute(sql, (name,))
    result = cursor.fetchone()
    return result[0] if result else None

def parse_cat_file(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return root

def ingest_data(root, cursor):
    factions = {}

    for faction in root.findall(".//catalogueLinks/catalogueLink"):
        faction_name = faction.get('name')
        faction_description = faction.get('id')  # Or any other attribute that fits as description
        insert_faction(cursor, faction_name, faction_description)
        factions[faction_name] = get_faction_id(cursor, faction_name)

    for unit in root.findall(".//sharedSelectionEntries/selectionEntry"):
        unit_name = unit.get('name')
        unit_description = unit.get('id')  # Or any other attribute that fits as description
        faction_name = None
        for categoryLink in unit.findall(".//categoryLinks/categoryLink"):
            if "Faction" in categoryLink.get('name'):
                faction_name = categoryLink.get('name').split(": ")[1]
                break
        if faction_name and faction_name in factions:
            faction_id = factions[faction_name]
            insert_unit(cursor, faction_id, unit_name, unit_description)

def main():
    db_connection = connect_to_db()
    cursor = db_connection.cursor()

    root = parse_cat_file('path_to_your_cat_file.cat')
    ingest_data(root, cursor)

    db_connection.commit()
    cursor.close()
    db_connection.close()
